In [ ]:
#install pyspark library
!pip install pyspark

In [ ]:
#import pyspark
import pyspark

In [ ]:
#import sparksession 
from pyspark.sql import SparkSession

In [ ]:
#creating a sparksession object and providing appName 
spark=SparkSession.builder.appName("optimization").getOrCreate()

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.option("header", "true").csv("/Users/krishnapratap/Desktop/partation/data/*")

# use take() in place of collect() for reduce time


In [ ]:
%time AirlineDF.take(5)

In [ ]:
%time AirlineDF.show(5)

In [ ]:
# calculate total no. of flight by Unique Carriers
%time AirlineDF.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
AirlineDF.registerTempTable("AirlineTable")

In [ ]:
#how to see all columns with datatype
spark.sql("describe AirlineTable").show()

In [ ]:
# calculate total no. of flight by Unique Carriers
%time spark.sql("select UniqueCarrier,count(UniqueCarrier) from AirlineTable group by UniqueCarrier").show()

In [ ]:
#calculate total columns in dataframe
len(AirlineDF.columns)

# Follow process for optimization

# When data is huge otherwise not

# use coalesce() in place of repartition() to reduce the no. of partition 

# Process:- 1 reduce no. of partition

In [ ]:
#to check how many partation in current AirlineDF dataframe
AirlineDF.rdd.getNumPartitions()

In [ ]:
#Not Use repartition for reduce no. of partation
#AirlineDF1 = AirlineDF.repartition(2)

In [ ]:
#Reduce No. of partation from 22 to 4
AirlineDF1 = AirlineDF.coalesce(4)

In [ ]:
AirlineDF1.rdd.getNumPartitions()

In [ ]:
#Again Check processing time
# calculate total no. of flight by Unique Carriers
%time AirlineDF1.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
AirlineDF1.registerTempTable("AirlineTable1")

In [ ]:
# calculate total no. of flight by Unique Carriers
%time spark.sql("select UniqueCarrier,count(UniqueCarrier) from AirlineTable1 group by UniqueCarrier").show()

# Apache Parquet is a columnar file format that provides optimizations to speed up queries and is a far more efficient file format than CSV or JSON, supported by many data processing systems.

# Process:- 2 write parquet file and create new dataframe for parquet file

In [ ]:
# first write dataframe into parquet file
AirlineDF1.write.parquet("parquet/")

In [ ]:
# Now create new dataframe from parquet file
AirlineDF_Par = spark.read.parquet("parquet/*")

In [ ]:
AirlineDF_Par.take(5)

In [ ]:
#Again Check processing time created dataframe from parquet file
# calculate total no. of flight by Unique Carriers
%time AirlineDF_Par.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
AirlineDF_Par.registerTempTable("AirlineTable2")

In [ ]:
# calculate total no. of flight by Unique Carriers
%time spark.sql("select UniqueCarrier,count(UniqueCarrier) from AirlineTable2 group by UniqueCarrier").show()

In [ ]:
%time spark.sql("select rtrim(DestCityName) from AirlineTable2").show()

In [ ]:
%time spark.sql("select substring_index(DestCityName, ',',1) as DestCityName from AirlineTable2").show()

# avoid PySpark UDF’s and use Spark SQL built-in functions as these functions provide optimization 

In [ ]:
Flight_Destination = AirlineDF.select("DestAirportID", "DestAirportSeqID", "Dest", 
                                      "DestCityName","DestState","DestStateName")

In [ ]:
Flight_Destination.show()

In [ ]:
from pyspark.sql.functions import udf

In [ ]:
from pyspark.sql.functions import col

In [ ]:
def destination(str):
    resStr=""
    arr = str.split(",")
    for x in arr:
       resStr= arr[0]
    return resStr 

In [ ]:
#Converting python function to UDF
destinationUDF = udf(lambda z: destination(z))

In [ ]:
#before udf apply
Flight_Destination.select("Dest","DestCityName").show()

In [ ]:
#when udf apply
%time Flight_Destination.select(col("Dest"),destinationUDF(col("DestCityName")).alias("DestCityName") ).show(truncate=False)

# Avoid UDF use spark sql

In [ ]:
Flight_Destination.registerTempTable("AirlineTable3")

In [ ]:
%time spark.sql("select Dest,substring_index(DestCityName, ',',1) as DestCityName from AirlineTable3").show()

# broadcast variables are used to save the copy of data across all nodes

In [ ]:
from pyspark import SparkContext, broadcast
from pyspark.sql import SparkSession 
import pyspark.sql.functions as func

In [ ]:
sc = spark.sparkContext

In [ ]:
words_new = sc.broadcast(["AA"]) 

In [ ]:
#call brodcast by using value
data = words_new.value # accessing the value stored in the broadcast in master

In [ ]:
data

In [ ]:
%time AirlineDF_Par.select("Year","UniqueCarrier").show()

In [ ]:
# without Broadcast variable on filter
%time AirlineDF_Par.where((AirlineDF_Par['UniqueCarrier']).isin('AA')).count()

In [ ]:
# Broadcast variable on filter
%time AirlineDF_Par.where((AirlineDF_Par['UniqueCarrier']).isin(data)).count()

#  accumulator variables are used for the information through associative and cummulative operations

In [ ]:
#create sparkcontext
from pyspark.context import SparkContext
sc = SparkContext()

In [ ]:
#creating a accumulator variable
accum=sc.accumulator(0.5)

In [ ]:
#empty variable
accum.value

In [ ]:
#create RDD
RDD=sc.parallelize([10,20,30,40,50])

In [ ]:
RDD.foreach(lambda x:accum.add(x))

In [ ]:
accum.value

In [ ]:
RDD.foreach(lambda x:accum.add(x))

In [ ]:
accum.value

# serialization in pyspark

# Serialization is used for performance tuning on Apache Spark

PySpark supports custom serializers for transferring data; this can improve
performance.

By default, PySpark uses L{PickleSerializer} to serialize objects using Python's
C{cPickle} serializer, which can serialize nearly any Python object.
Other serializers, like L{MarshalSerializer}, support fewer datatypes but can be
faster.

The serializer is chosen when creating L{SparkContext}:

In [ ]:
spark.stop()

In [ ]:
from pyspark.context import SparkContext
from pyspark.serializers import MarshalSerializer
sc = SparkContext("local", "serialization app", batchSize=2, serializer = MarshalSerializer())

In [ ]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
spark.sparkContext is sc

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.parquet("parquet/*")

In [ ]:
# calculate total no. of flight by Unique Carriers
%time AirlineDF.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
#check no. of partition
AirlineDF.rdd.getNumPartitions()

In [ ]:
spark.stop()

In [ ]:
from pyspark.context import SparkContext
from pyspark.serializers import PickleSerializer
sc = SparkContext("local", "serialization app", batchSize=2, serializer=PickleSerializer())

In [ ]:
spark = SparkSession.builder.enableHiveSupport().getOrCreate()
spark.sparkContext is sc

In [ ]:
#To create dataframe form External datasets
AirlineDF = spark.read.parquet("parquet/*")

In [ ]:
# calculate total no. of flight by Unique Carriers
%time AirlineDF.select("UniqueCarrier").groupby("UniqueCarrier").count().show()

In [ ]:
#check no. of partition
AirlineDF.rdd.getNumPartitions()

# spark Parallelism

spark.default.parallelism is the default number of partition set by spark which is by default 200. and if you want to increase the number of partition than you can apply the property


One important way to increase parallelism of spark processing is to increase the number of executors on the cluster. However, knowing how the data should be distributed, so that the cluster can process data efficiently is extremely important. The secret to achieve this is partitioning in Spark.

# Note:- http://spark-configuration.luminousmen.com/

# for RDD use spark.default.parallelism

In [ ]:
#creating a sparksession object and providing appName 
spark0=SparkSession.builder.appName("optimization0").getOrCreate()

In [ ]:
spark0.conf.set("spark.default.parallelism",1)

In [ ]:
sc1 = spark0.sparkContext

In [ ]:
rdd2=sc1.textFile("/Users/krishnapratap/Desktop/partation/data/*")

In [ ]:
sc1.defaultParallelism

In [ ]:
# Split words using flatMap
rdd_word1 = rdd2.flatMap(lambda x: x.split(","))

In [ ]:
# Create a paired-rdd
rdd_pair1 = rdd_word1.map(lambda x: (x, 1))

In [ ]:
# Count occurence per word using reducebykey()
rdd_reduce1 = rdd_pair1.reduceByKey(lambda x,y: x+y)
%time rdd_reduce1.collect()

# for dataframe use spark.sql.shuffle.partitions

In [ ]:
#creating a sparksession object and providing appName 
spark1=SparkSession.builder.appName("optimization1").getOrCreate()

In [ ]:
spark1.conf.set("spark.sql.shuffle.partitions",50)

In [ ]:
#To create dataframe form External datasets
AirlineDF5 = spark1.read.option("header", "true").csv("/Users/krishnapratap/Desktop/partation/data/*")

In [ ]:
# calculate total rows in dataframe
%time AirlineDF5.select("UniqueCarrier").groupby("UniqueCarrier").count().show()